In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pip install --upgrade -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


# caustique d'onde

In [3]:
import os
from IPython.display import display
import numpy as np
import MotionClouds as mc
from caustique import init, Caustique
figpath = '2022-07-25_caustique'

In [4]:
print(f'Saving our simulations in={figpath}')

Saving our simulations in=2022-07-25_caustique


In [5]:
opt = init()
opt.figpath = figpath

In [6]:
opt

Namespace(tag='caustique', figpath='2022-07-25_caustique', vext='mp4', nx=256, ny=256, nframe=160, bin_dens=2, bin_spectrum=6, seed=42, H=6.0, variation=0.4, sf_0=0.006, B_sf=0.002, V_Y=0.3, V_X=0.3, B_V=4.0, theta=2.399988291783386, B_theta=1.0471975511965976, min_lum=0.2, gamma=2.4, fps=18, multispectral=True, cache=False, verbose=False)

# une simple caustique

## a wave pattern

In [7]:
c = Caustique(opt)
wave_videoname = f'{opt.figpath}/{opt.tag}_wave'
if not os.path.isfile(f'{wave_videoname}{c.mc.vext}'):
    print(f'Doing {wave_videoname}{c.mc.vext}')
    z = c.wave()
    c.mc.anim_save(z.swapaxes(0, 1), wave_videoname)
c.show(f'{wave_videoname}{c.mc.vext}')

## generating caustics

In [8]:
c = Caustique(opt)
z = c.wave()
z.shape

(256, 256, 160)

In [9]:
N_wavelengths = len(c.cs_srgb.cmf[:, 0])
N_wavelengths, N_wavelengths//c.opt.bin_spectrum

(81, 13)

In [10]:
for ii, i in enumerate(range(0, N_wavelengths, c.opt.bin_spectrum)): print(ii, i)

0 0
1 6
2 12
3 18
4 24
5 30
6 36
7 42
8 48
9 54
10 60
11 66
12 72
13 78


In [11]:
videoname = f'{opt.figpath}/{opt.tag}.{opt.vext}'
if not os.path.isfile(videoname):
    c = Caustique(opt)
    z = c.wave()
    videoname = c.plot(z, videoname=videoname)

In [12]:
c.show(videoname)

In [13]:
z.shape

(256, 256, 160)

## exploring parameters

In [14]:
N_scan = 9
base = 2

In [15]:
opt = init()
opt.figpath = figpath
opt.H = 10.
opt.variation = .09

c = Caustique(opt)
# compute just once
z = c.wave()

for H_ in c.opt.H*np.logspace(-1, 1, N_scan, base=base):
    opt = init()
    opt.figpath = figpath
    c = Caustique(opt)

    print(f'H = {H_:.3f}')
    c.opt.H = H_
    c.opt.tag = f'{opt.tag}_H_{H_:.3f}'
    videoname = f'{opt.figpath}/{c.opt.tag}.{opt.vext}'
    print(c.opt.figpath, videoname)
    if not os.path.isfile(videoname):
        url=c.plot(z, videoname=videoname)
    display(c.show(videoname))

H = 5.000
2022-07-25_caustique 2022-07-25_caustique/caustique_H_5.000.mp4


H = 5.946
2022-07-25_caustique 2022-07-25_caustique/caustique_H_5.946.mp4


H = 7.071
2022-07-25_caustique 2022-07-25_caustique/caustique_H_7.071.mp4


H = 8.409
2022-07-25_caustique 2022-07-25_caustique/caustique_H_8.409.mp4


H = 10.000
2022-07-25_caustique 2022-07-25_caustique/caustique_H_10.000.mp4


H = 11.892
2022-07-25_caustique 2022-07-25_caustique/caustique_H_11.892.mp4


H = 14.142
2022-07-25_caustique 2022-07-25_caustique/caustique_H_14.142.mp4


H = 16.818
2022-07-25_caustique 2022-07-25_caustique/caustique_H_16.818.mp4


H = 20.000
2022-07-25_caustique 2022-07-25_caustique/caustique_H_20.000.mp4


In [16]:
opt = init()
opt.figpath = figpath
opt.H = 10.
opt.variation = .09

c = Caustique(opt)
# compute just once
z = c.wave()

for variation_ in opt.variation*np.logspace(-1, 1, N_scan, base=10, endpoint=True):
    opt = init()
    opt.figpath = figpath
    c = Caustique(opt)

    print(f'variation = {variation_:.3f}')
    c.opt.variation = variation_
    c.opt.tag = f'{opt.tag}_variation_{variation_:.3f}'
    videoname = f'{opt.figpath}/{c.opt.tag}.{opt.vext}'
    if not os.path.isfile(videoname):
        url=c.plot(z, videoname=videoname)
    display(c.show(videoname))

variation = 0.009


variation = 0.016


variation = 0.028


variation = 0.051


variation = 0.090


variation = 0.160


variation = 0.285


variation = 0.506


variation = 0.900


In [17]:
for variable in ['sf_0', 'B_sf', 'theta', 'B_theta', 'V_X', 'V_Y', 'B_V']:
    print(f'======{variable}======')
    for modul in np.logspace(-1, 1, N_scan, base=base):
        opt = init()
        opt.figpath = figpath

        c = Caustique(opt)
        c.d[variable] *= modul
        c.opt.tag = f'{opt.tag}_{variable}_modul_{modul:.3f}'
        videoname = f'{opt.figpath}/{c.opt.tag}.{opt.vext}'

        print(f'{variable}={variable}(default)*{modul:.3f}={c.d[variable]:.3E}')
        if not os.path.isfile(videoname):
            print('Doing ', videoname)
            z = c.wave()
            mcname = f'{opt.figpath}/{c.opt.tag}'
            if not os.path.isfile(f'{mcname}{mc.vext}'): 
                print('Doing ', f'{mcname}{mc.vext}')
                mc.anim_save(z.swapaxes(0, 1), f'{mcname}')
            url=c.plot(z, videoname=videoname)
        display(c.show(videoname))

======sf_0======
sf_0=sf_0(default)*0.500=3.000E-03


sf_0=sf_0(default)*0.595=3.568E-03


sf_0=sf_0(default)*0.707=4.243E-03


sf_0=sf_0(default)*0.841=5.045E-03


sf_0=sf_0(default)*1.000=6.000E-03


...

B_sf=B_sf(default)*0.841=1.682E-03


B_sf=B_sf(default)*1.000=2.000E-03


B_sf=B_sf(default)*1.189=2.378E-03


B_sf=B_sf(default)*1.414=2.828E-03
Doing  2022-07-25_caustique/caustique_B_sf_modul_1.414.mp4
Doing  2022-07-25_caustique/caustique_B_sf_modul_1.414.mp4
ffmpeg version 5.0.1 Copyright (c) 2000-2022 the FFmpeg developers
built with Apple clang version 13.1.6 (clang-1316.0.21.2.5)
configuration: --prefix=/usr/local/Cellar/ffmpeg/5.0.1_3 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb

KeyboardInterrupt: 